In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
from concurrent.futures import ThreadPoolExecutor, as_completed

In [9]:
# Paths for input file, lookup shapefile, and output folder
input_txt_file = r"D:\RISHIN\01_50m\combined_shapefile\USWF_Mapping_Task\File_Name.txt"
lookup_shapefile= r"D:\RISHIN\01_50m\lookup_file\finalmod3.gpkg"
output_folder = r"D:\RISHIN\01_50m\out_csv_new"

In [10]:
# Function to process the file name and obtain the longitude and latitude
def process_file_name(file_name):
    parts = file_name.split("_")
    third_word = parts[2]
    first_num = int(third_word[0])
    long = int(third_word[1:3])
    lat = int(third_word[3:5])
    
    # Ensure the longitude is negative for the USA region
    if first_num == 3:
        return -long, lat
    elif first_num == 4:
        return -100 - long, lat
    else:
        raise ValueError("Unexpected first number in the third word")

# Function to create a polygon grid in a 100 km² area with 50 m spacing, starting from lower-right corner size 1 dgre=100km
def create_polygon_grid(lower_right_long, lower_right_lat, grid_size=1, spacing=0.0005):
    num_points = int(grid_size / spacing)
    centroids = []

    for i in range(num_points):
        for j in range(num_points):
            lr_long = round(lower_right_long - (i * spacing),5)
            lr_lat = round(lower_right_lat + (j * spacing),5)
            centroid_long = round(lr_long - (spacing / 2),5)
            centroid_lat = round(lr_lat + (spacing / 2),5)
            centroid = Point(centroid_long, centroid_lat)
            centroids.append(centroid)

    centroid_gdf = gpd.GeoDataFrame(geometry=centroids, crs="EPSG:4326")

    return  centroid_gdf

# Define the main function to process each file
def batch_process(file_names, lookup_shapefile, output_folder):
    # Load the lookup shapefile
    lookup_gdf = gpd.read_file(lookup_shapefile,engine='pyogrio').to_crs("EPSG:4326")

    for file_name in file_names:
        try:
            # Process the filename to extract longitude and latitude
            long, lat = process_file_name(file_name)
            
            # Create the polygon grid and centroids from the lower-right corner
            centroid_gdf = create_polygon_grid(long, lat)
            
            # Perform the spatial join to find which centroids intersect the lookup shapefile
            joined_gdf = gpd.sjoin(centroid_gdf, lookup_gdf, how="left", predicate="intersects")
            # Extract Longitude and Latitude from the geometry column
            joined_gdf["Longitude"] = joined_gdf.geometry.x
            joined_gdf["Latitude"] = joined_gdf.geometry.y
            joined_gdf=joined_gdf.drop(columns={"geometry","index_right"})
            # Reorder the columns to have 'Longitude' and 'Latitude' first
            joined_gdf = joined_gdf[["Longitude", "Latitude"] + [col for col in joined_gdf.columns if col not in ["Longitude", "Latitude"]]]
        
            
            # Define the output CSV file path with filename suffix
            output_csv_file = os.path.join(output_folder, file_name.replace('.DAT','_lookup.csv'))
            
            # Save the joined result as a CSV file
            joined_gdf.to_csv(output_csv_file, index=False)
            print(f"Processed and saved: {output_csv_file}")
        
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

In [11]:
with open(input_txt_file, 'r') as f:
    file_names = f.read().splitlines()
os.makedirs(output_folder, exist_ok=True)

In [12]:
batch_process(file_names, lookup_shapefile, output_folder)


Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36545_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36644_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36645_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36646_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36647_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36744_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36745_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36746_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36747_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36748_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36843_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36844_lookup.csv
Processed and saved: D:\RISHIN\01_50m\out_csv_new\FUEL_US_36845_lookup.csv
Processed and saved: D:\R